In [ ]:
# get IQR
for i,num_col in enumerate(num_cols):

    globals()[f"{num_col}_IQR"] = 2*(data[num_col].quantile(.75) - data[num_col].quantile(.25))
    print(i+1, f"| {num_col}_IQR =", globals()[f"{num_col}_IQR"])

In [ ]:
outlier_col_name = list(num_cols.copy())

for i,col_name in enumerate(outlier_col_name):
    outlier_col_name[i] = f"is_OutLier_{col_name}"
    
outlier_col_name

In [ ]:
%%time
# Detect Outlier

# numpy 계산 : 207ms
# def 호출 numpy 계산 : 443ms
# 1차 loop 에서 반복 최소화 : 61ms
# Vectorize 로 복잡한 연산(함수를 통한 전처리)은 해봤는데, 잘 안되던데.. 어떻게 하는거지

is_Outlier= np.zeros((len(data),len(num_cols)))
###
train = data[num_cols]
###

def check_outlier(original_np,  outlier_np,  low_cut,  high_cut,  col_id):
    
    
    for i in range(len(original_np)):

        if original_np[i,col_id] < low_cut :
                original_np[i,col_id] = low_cut
                outlier_np[i,col_id] = 1

        if original_np[i,col_id] > high_cut :
                original_np[i,col_id] = high_cut
                outlier_np[i,col_id] = 1
 
    return original_np[:,col_id], outlier_np[:,col_id]
    


for col_id, col in enumerate(num_cols):
    low_cut = train[col].quantile(.25) - globals()[f"{col}_IQR"]
    high_cut = train[col].quantile(.75) + globals()[f"{col}_IQR"]

    train.loc[:,col], is_Outlier[:,col_id] = check_outlier(train.to_numpy(),is_Outlier,low_cut,high_cut,col_id)
            
        
is_Outlier = pd.DataFrame(is_Outlier, columns = outlier_col_name)
train_OutL = pd.concat([train,is_Outlier],axis=1)
train_OutL